In [1]:
import pandas as pd
from SwissArmyKnife import YTClient
from datetime import datetime
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [45]:
ytc = YTClient('Hahn')
data_hahn = {
    'consumption' : '/home/cloud_analytics/tmp/artkaz/cons_clustering',
    'ba_info': '/home/cloud_analytics/tmp/artkaz/ba_info',
    'antifraud': '/home/cloud_analytics/tmp/artkaz/antifraud_bad_accounts_2019_06_04', #https://yql.yandex-team.ru/Operations/XPeS4Gim9UVcUXTm9eZE_WT1xY1WGIyqCZ0vCY6DxFQ=
    'destination': '/home/cloud_analytics/tmp/artkaz/similar_consumption_patterns_2019_06_05'
}


Мы хотим найти кластеры, очень близкие по потреблению. Используем данные начиная с 1го апреля за первые полные пять дней потребления (самый первый день не включаем)

In [4]:
# тут лежат результаты запроса https://yql.yandex-team.ru/Operations/XOGISGim9Y_WMu-K7n1siU4GXolBOcqSfZu4QFOr6bU=
# данные о потреблении начиная с 1 апреля
data = ytc.read_yt_table(data_hahn['consumption'])

In [5]:
data.shape

(9736155, 4)

In [6]:
data.head()

,billing_account_id,cost,date,sku
0,dn2004tsaahbl68oadh2,71.702045,1554076800,compute.vm.cpu.c100
1,dn2004tsaahbl68oadh2,49.885903,1554076800,compute.vm.ram
2,dn2004tsaahbl68oadh2,129.680868,1554076800,marketplace.windows.cpu.c100
3,dn2004tsaahbl68oadh2,8.686293,1554076800,nbs.network-nvme.allocated
4,dn2004tsaahbl68oadh2,0.000000,1554076800,network.egress.inet


Агрегируем данные, убираем аккаунты, потребляющие менее пяти дней, оставляем данные за первые 5 дней потребления:

In [7]:
consumption_start_dates = data[['billing_account_id','date','cost']].\
                            groupby('billing_account_id',as_index=False).agg({ 
                                    'date':'min'}).rename({'date':'first_date'}, axis=1)


In [8]:
daily_consumption = data[['billing_account_id',
                          'date',
                          'cost']].groupby(['billing_account_id', 
                                            'date'],as_index=False).agg({ 
                                                                        'cost':'sum'})

In [9]:
daily_consumption.head()

,billing_account_id,date,cost
0,dn2004tsaahbl68oadh2,1553990400,1104.121555
1,dn2004tsaahbl68oadh2,1554076800,1050.223432
2,dn2004tsaahbl68oadh2,1554163200,34.739543
3,dn2004tsaahbl68oadh2,1554249600,34.739141
4,dn2004tsaahbl68oadh2,1554336000,34.739141


In [10]:
daily_consumption2 = pd.merge(daily_consumption, consumption_start_dates, how='left', on='billing_account_id')

In [11]:
daily_consumption2.head()

,billing_account_id,date,cost,first_date
0,dn2004tsaahbl68oadh2,1553990400,1104.121555,1553990400
1,dn2004tsaahbl68oadh2,1554076800,1050.223432,1553990400
2,dn2004tsaahbl68oadh2,1554163200,34.739543,1553990400
3,dn2004tsaahbl68oadh2,1554249600,34.739141,1553990400
4,dn2004tsaahbl68oadh2,1554336000,34.739141,1553990400


In [12]:
daily_consumption2.shape

(1020078, 4)

In [13]:
#Выбираем апрельское потребление

daily_consumption2 = daily_consumption2[daily_consumption2['first_date']>=1554076800] #2019-04-01

In [14]:
daily_consumption2.shape

(357186, 4)

In [15]:
daily_consumption2['day_number_int'] = (daily_consumption2['date'] - daily_consumption2['first_date'])/86400. 

daily_consumption2['day_number'] = daily_consumption2['day_number_int'].apply(
    lambda x: 'day_0' + str(int(x)) if x<10 else 'day_' + str(int(x)))

In [16]:
cons_max_dates = daily_consumption2.groupby('billing_account_id', as_index=False).agg({'day_number_int':'max'})

In [17]:
ba_mature = cons_max_dates[cons_max_dates['day_number_int']>=5]

In [18]:
ba_mature.head()

,billing_account_id,day_number_int
0,dn2005pvulgj42lae673,23.0
1,dn2007crg75ggb75s88f,22.0
2,dn200935k71qhasmfjn0,11.0
4,dn200ae4mtfsog16t9gv,11.0
5,dn200jrmfioingtd47v4,19.0


In [19]:
daily_consumption3 = pd.merge(daily_consumption2, 
                              ba_mature['billing_account_id'], 
                              how='inner', 
                              on='billing_account_id')

In [20]:
daily_consumption4 = daily_consumption3[daily_consumption3['day_number_int'] <= 5]

In [21]:
daily_consumption4.head()

,billing_account_id,date,cost,first_date,day_number_int,day_number
0,dn2005pvulgj42lae673,1557619200,7.315200,1557619200,0.0,day_00
1,dn2005pvulgj42lae673,1558051200,0.609600,1557619200,5.0,day_05
9,dn2007crg75ggb75s88f,1557792000,1086.562494,1557792000,0.0,day_00
10,dn2007crg75ggb75s88f,1557878400,1649.405217,1557792000,1.0,day_01
11,dn2007crg75ggb75s88f,1557964800,1725.248607,1557792000,2.0,day_02


In [22]:
daily_consumption4.shape

(63065, 6)

"Разворачиваем" данные в пивотку:

In [23]:
daily_consumption_pivot = daily_consumption4.pivot(
    index='billing_account_id', 
    columns='day_number', 
    values='cost').fillna(0)



In [24]:
daily_consumption_pivot.head()

day_number,day_00,day_01,day_02,day_03,day_04,day_05
billing_account_id,,,,,,
dn2005pvulgj42lae673,7.315200,0.000000,0.000000,0.000000,0.000000,0.609600
dn2007crg75ggb75s88f,1086.562494,1649.405217,1725.248607,1725.248607,1725.248607,1222.051097
dn200935k71qhasmfjn0,23.410420,75.843894,75.842221,75.843894,75.830550,75.835884
dn200ae4mtfsog16t9gv,347.239539,512.506254,474.181961,11.118445,11.120375,11.118413
dn200jrmfioingtd47v4,1424.865318,2050.028877,1452.103788,1025.014438,423.179774,14.824599


In [25]:
daily_consumption_pivot.shape

(10809, 6)

Мы хотим найти кластеры с максимальной плотностью точек, поэтому тут подойдет алгоритм MeanShift:

In [26]:
from sklearn.cluster import MeanShift
import numpy as np
clustering = MeanShift(bandwidth=2).fit(daily_consumption_pivot.loc[:, 'day_01':'day_05'])
clustering.labels_

array([   0, 1140,   92, ...,    5, 3470, 1735])

In [27]:
daily_consumption_pivot['cluster'] = clustering.labels_

In [28]:
clusters_count = daily_consumption_pivot.groupby('cluster', as_index=False).agg({'day_00':'count'}).rename({'day_00':'c'}, axis=1)


Всего получилось около 100 кластеров, с 10 и более биллинг-аккаунтами, у которых практически одинаковое потребление за первые пять дней:

In [29]:
clusters_count[clusters_count['c']>10].shape

(150, 2)

Добавляем данные о ба, считаем среднее потребление за 5 полных дней:

In [30]:
ba_info = ytc.read_yt_table(data_hahn['ba_info'])
#https://yql.yandex-team.ru/Operations/XOO3wWHljjB8iKvwb2h8SzJ4E4oFbAIJnOstaMX4LKg=

In [31]:
daily_consumption_pivot2 = pd.merge(daily_consumption_pivot, ba_info, how='left', on='billing_account_id')

In [32]:
daily_consumption_pivot2['avg_consumption'] = daily_consumption_pivot2.loc[:,'day_01':'day_05'].apply(np.mean, axis=1)

Наложим на полученные кластеры результаты антифрода:

In [39]:
antifraud_data = ytc.read_yt_table(data_hahn['antifraud'])[['billing_account_id', 'is_bad']]

In [40]:
antifraud_data.head()

,billing_account_id,is_bad
0,dn2r31h8ihg6getjv4ar,False
1,dn2bd8k1jred3taus4lb,False
2,dn28h2s9i5nm5i343dd4,False
3,dn2p72ldg8g49du21ket,False
4,dn2j3sdl22e6duu274iu,False


In [41]:
daily_consumption_pivot3 = pd.merge(daily_consumption_pivot2, antifraud_data, how='left', on='billing_account_id')

In [42]:
daily_consumption_pivot3.head()

,billing_account_id,day_00,day_01,day_02,day_03,day_04,day_05,cluster,block_reason,created_at,...,passport_email,passport_phone,payment_type,person_type,registration_ip,state,type,usage_status,avg_consumption,is_bad
0,dn2005pvulgj42lae673,7.315200,0.000000,0.000000,0.000000,0.000000,0.609600,0,None,1.557689e+09,...,undefer@gmail.com,79652171913,card,individual,78.31.74.4,active,self-served,trial,0.121920,False
1,dn2007crg75ggb75s88f,1086.562494,1649.405217,1725.248607,1725.248607,1725.248607,1222.051097,1140,None,1.557813e+09,...,mikasdomert@yandex.ru,79175609663,card,individual,2a02:6b8:c00:2d91:0:4362:a556:ab6e,active,self-served,trial,1609.440427,False
2,dn200935k71qhasmfjn0,23.410420,75.843894,75.842221,75.843894,75.830550,75.835884,92,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.839289,False
3,dn200ae4mtfsog16t9gv,347.239539,512.506254,474.181961,11.118445,11.120375,11.118413,69,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204.009090,True
4,dn200jrmfioingtd47v4,1424.865318,2050.028877,1452.103788,1025.014438,423.179774,14.824599,52,manual,1.558065e+09,...,trofimvyrypaev@yandex.ru,79263074644,card,individual,5.166.88.29,suspended,self-served,trial,993.030295,True


In [43]:
daily_consumption_pivot3['is_bad'].sum()

4605

In [50]:
daily_consumption_pivot3['is_bad'] = daily_consumption_pivot3['is_bad'].apply(lambda x: str(x))
ytc.write_to_yt(df=daily_consumption_pivot3.fillna('').sort_values('cluster'), 
                yt_path=data_hahn['destination'])

Посчитаем статистику по кластерам: 
    * количество аккаунтов,
    * среднее потребление, 
    * количество активных и заблокированных аккаунтов, 
    * отношение фрода к заблокированным

In [47]:
pd.set_option('display.max_rows', 1000)

In [48]:
 def count_categories(categories):
    res = {}
    for c in categories:
        if c in res:
            res[c] += 1
        else:
            res[c] = 1
    
    return res
    

clusters_stat = daily_consumption_pivot3.groupby('cluster', as_index=False).agg({
                                                 'avg_consumption':'mean',
                                                 'billing_account_id':'count',
                                                 'state': count_categories,
                                                 'block_reason': count_categories,
                                                 'is_bad': lambda x: count_categories(x)['True'] if 'True' in count_categories(x) else 0
    
    
                                                                                },)
clusters_stat['is_bad'] = clusters_stat['is_bad'].apply(int)

clusters_stat['active_count'] = clusters_stat['state'].apply(lambda s: s['active'] if 'active' in s else 0)
clusters_stat['suspended_count'] = clusters_stat['state'].apply(lambda s: s['suspended'] if 'suspended' in s else 0)

clusters_stat['active_pct'] = clusters_stat['active_count'] / clusters_stat['billing_account_id']
clusters_stat['suspended_pct'] = clusters_stat['suspended_count'] / clusters_stat['billing_account_id']

clusters_stat['fraud_to_suspended'] = clusters_stat['is_bad'] / clusters_stat['suspended_count']
clusters_stat['fraud_to_suspended'] = clusters_stat['is_bad'] / clusters_stat['billing_account_id']

clusters_stat = clusters_stat[(clusters_stat['billing_account_id'] >= 10) & 
                              (clusters_stat['avg_consumption']> 100)  
                              #(clusters_stat['active_pct']> 0.2)  ]
                             ]


clusters_to_look = set(clusters_stat['cluster']) 
clusters_stat.sort_values('fraud_to_suspended', ascending=False)

,cluster,billing_account_id,state,avg_consumption,is_bad,block_reason,active_count,suspended_count,active_pct,suspended_pct,fraud_to_suspended
78,78,21,"{u'active': 20, nan: 1}",196.811953,21,"{None: 20, nan: 1}",20,0,0.952381,0.000000,1.000000
42,42,34,{u'suspended': 34},2050.904341,34,{u'manual': 34},0,34,0.000000,1.000000,1.000000
44,44,35,{u'suspended': 35},1259.042794,35,{u'manual': 35},0,35,0.000000,1.000000,1.000000
45,45,33,{u'suspended': 33},1431.393648,33,{u'manual': 33},0,33,0.000000,1.000000,1.000000
46,46,32,{u'suspended': 32},366.896201,32,{u'manual': 32},0,32,0.000000,1.000000,1.000000
48,48,32,{u'suspended': 32},145.477192,32,{u'manual': 32},0,32,0.000000,1.000000,1.000000
149,149,11,{u'suspended': 11},507.397171,11,{u'manual': 11},0,11,0.000000,1.000000,1.000000
148,148,11,{u'suspended': 11},659.644114,11,{u'manual': 11},0,11,0.000000,1.000000,1.000000
51,51,31,{u'active': 31},297.084083,31,{None: 31},31,0,1.000000,0.000000,1.000000
52,52,30,{u'suspended': 30},993.050306,30,{u'manual': 30},0,30,0.000000,1.000000,1.000000


In [49]:
clusters_stat.head()

,cluster,billing_account_id,state,avg_consumption,is_bad,block_reason,active_count,suspended_count,active_pct,suspended_pct,fraud_to_suspended
1,1,274,"{u'active': 2, u'suspended': 272}",744.527575,233,"{None: 1, u'manual': 213, u'trial_expired': 60}",2,272,0.007299,0.992701,0.850365
2,2,210,{u'suspended': 210},404.829478,185,{u'manual': 210},0,210,0.000000,1.000000,0.880952
3,3,162,{u'suspended': 162},1725.285754,16,"{u'manual': 65, u'trial_expired': 97}",0,162,0.000000,1.000000,0.098765
5,5,104,"{nan: 73, u'active': 31}",431.299826,0,"{nan: 73, None: 31}",31,0,0.298077,0.000000,0.000000
7,7,101,{u'suspended': 101},298.428673,85,{u'manual': 101},0,101,0.000000,1.000000,0.841584


Можно сделать следующие выводы:
    1. Есть кластеры, которые полностью заблочены и полностью определены антифродом (fraud_to_suspended = 1). Большое кол-во таких кластеров - хороший показатель работы антифрода
    2. Есть кластеры, которые покрыты антифродом частично. (fraud_to_suspended between (0,1)). Основные причины блокировки - кончилсяч триал или отвязал карту - возможно эти аккаунты тоже фрод, который истратил свой триал и ушел. Нужно проверять
    3. Кластеры, где fraud_to_suspended=0 or NaN - тут фрод маловероятен, потому что не определено хотя бы одного аккаунта и практически нет или вообще отсустсвуют аккаунты, заблокированные по другим причинам - большинство аккаунтов в этой группе активные
    
 В итоге, я бы советовал обратить внимание на аккаунты из группы 2. 
 Все данные я выложил на YT в таблицу /home/cloud_analytics/tmp/artkaz/similar_consumption_patterns

In [180]:
print daily_consumption_pivot3.shape
print daily_consumption_pivot3.dropna().shape

(8240, 21)
(4721, 21)


In [181]:
daily_consumption_pivot3.dtypes

billing_account_id     object
day_00                float64
day_01                float64
day_02                float64
day_03                float64
day_04                float64
day_05                float64
cluster                 int64
block_reason           object
fraud_detected_by      object
name                   object
owner_id               object
passport_email         object
passport_phone         object
payment_type           object
person_type            object
registration_ip        object
state                  object
type                   object
avg_consumption       float64
is_bad                 object
dtype: object

In [182]:
daily_consumption_pivot3['is_bad'] = daily_consumption_pivot3['is_bad'].apply(lambda x: str(x))